# Embedding Models Experimentation (C.3)

Here several embedding models are trained and evaluated :)

In [ ]:
# Core libraries
import torch
import pandas as pd

# PyKEEN core
import pykeen
from pykeen.pipeline import pipeline
from pykeen.datasets import Nations
from pykeen.triples import TriplesFactory
import math

# Optional: prediction utilities (used only for link prediction tasks)
from pykeen import predict
device = "cuda" if torch.cuda.is_available() else "cpu"


/home/wtroiani/miniconda3/envs/sdm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# EVALUATION METRIC SCORE invented by me :)
# It's a weighted sum between evaluation metrics and complexity, so theres a tradeoff


def weighted_embedding_score(
    result_dict,
    embedding_dim,
    num_epochs,
    weights=None
):
    if weights is None:
        weights = {
            "hits_at_10": 0.4,
            "inverse_harmonic_mean_rank": 0.3,
            "median_rank": 0.2,
            "complexity": 0.1
        }

    hits10 = result_dict["hits_at_10"]
    ihmr = result_dict["inverse_harmonic_mean_rank"]
    median = result_dict["median_rank"]

    complexity = math.log(embedding_dim * num_epochs + 1)

    # Final score (higher is better)
    score = (
        weights["hits_at_10"] * hits10 +
        weights["inverse_harmonic_mean_rank"] * ihmr +
        weights["median_rank"] * (1 / (1 + median)) -
        weights["complexity"] * complexity
    )

    return {
        "score": score,
        "components": {
            "hits@10": hits10,
            "ihmr": ihmr,
            "1/(1+median)": 1 / (1 + median),
            "log_complexity": complexity
        }
    }


## Experiment 1: ???

In [ ]:
file_path = "../../data/graph_export.tsv"
tf = TriplesFactory.from_path(file_path)
training, testing = tf.split()

result = pipeline(
    training=training,
    testing=testing,
    model="TransE",
    model_kwargs=dict(
        embedding_dim=128,
    ),
    training_kwargs=dict(
        num_epochs=20
    ),
    optimizer_kwargs=dict(
        lr=0.01,
    ),
    negative_sampler_kwargs=dict(
        num_negs_per_pos=1,
    ),
    random_seed=2025,
    device = device
)

using automatically assigned random_state=2033177419
Training epochs on cpu: 100%|██████████| 20/20 [00:08<00:00,  2.22epoch/s, loss=0.267, prev_loss=0.275]
Evaluating on cpu:   0%|          | 0.00/2.00k [00:00<?, ?triple/s]WARNING:torch_max_mem.api:Encountered tensors on device_types={'cpu'} while only ['cuda'] are considered safe for automatic memory utilization maximization. This may lead to undocumented crashes (but can be safe, too).
Evaluating on cpu: 100%|██████████| 2.00k/2.00k [00:06<00:00, 289triple/s]
INFO:pykeen.evaluation.evaluator:Evaluation took 6.94s seconds


In [ ]:
evaluation = result.metric_results.to_dict()
print(evaluation['both']['realistic'])
print(weighted_embedding_score(result, ))

{'count': 4002.0,
 'inverse_geometric_mean_rank': 0.03148800879716873,
 'adjusted_inverse_harmonic_mean_rank': 0.15029553120341257,
 'median_absolute_deviation': 32.61724880712324,
 'adjusted_geometric_mean_rank_index': 0.976699575088844,
 'arithmetic_mean_rank': 240.5442352294922,
 'variance': 234525.546875,
 'adjusted_arithmetic_mean_rank_index': 0.8686105716136165,
 'geometric_mean_rank': 31.758119583129883,
 'standard_deviation': 484.27838134765625,
 'adjusted_arithmetic_mean_rank': 0.13186559798709938,
 'inverse_arithmetic_mean_rank': 0.004157239571213722,
 'inverse_harmonic_mean_rank': 0.15240496397018433,
 'inverse_median_rank': 0.0416666679084301,
 'harmonic_mean_rank': 6.561466070066029,
 'median_rank': 24.0,
 'z_arithmetic_mean_rank': 93.84115181764997,
 'z_geometric_mean_rank': 62.1077244663767,
 'z_inverse_harmonic_mean_rank': 441.3013791535653,
 'hits_at_1': 0.03073463268365817,
 'hits_at_3': 0.20814592703648177,
 'hits_at_5': 0.29485257371314344,
 'hits_at_10': 0.39005497

## Experiment 2: ???

## Experiment 3: ???

## Experiment 4: ???

## Experiment 5: ???